In [ ]:
# necessary library
import pandas as pd
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship, Subgraph
import csv

In [ ]:
# here is my pre-data-process, the raw data is composed of scenic spot entities and their various attributes.
# Then link each attribute to the spot entity as a triple, forming the following tri table below.

# tri = pd.concat([A,province,jingdian,zhulei,yalei,jibenlei,zhulei1,yalei1,jibenlei1,zhulei2,yalei2,jibenlei2], ignore_index=True)
# tri=tri.dropna()
# tri.to_csv("C:/Users/Taul/Desktop/tri.csv")

In [ ]:
# link to Neo4j
uri = "bolt://localhost:xxxx"  # your neo4j browser url
username = "neo4j"  # your user name
password = "xxx"  # your password
driver = GraphDatabase.driver(uri, auth=(username, password))

# define function for batch data import
def import_data_to_neo4j(driver, csv_file_path, batch_size=1000):
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)  # DictReader provides conviently read by column name
        batch = [1000]  
        count = 0 

        for row in reader:
            batch.append(row)
            count += 1

            if count % batch_size == 0:
                with driver.session() as session:
                    session.write_transaction(load_batch, batch)
                print(f"Processed {count} rows so far.")
                batch = []

        # import the last batch
        if batch:
            with driver.session() as session:
                session.write_transaction(load_batch, batch)
            print(f"Processed {count} rows in total.")

# define actual excution process
def load_batch(tx, batch):
    for row in batch:
        head = row['head']
        relation = row['relation']
        tail = row['tail']
        source = "import_2025_04"  

        tx.run("""
            MERGE (h:Entity {name: $head, source: $source})
            MERGE (t:Entity {name: $tail, source: $source})
            MERGE (h)-[:RELATES {type: $relation, source: $source}]->(t)
        """, head=head, relation=relation, tail=tail, source=source)

# 执行数据导入
csv_file_path = "C:/Users/Taul/Desktop/tri.csv"  # your CSV
import_data_to_neo4j(driver, csv_file_path)
